# Binary Classification Fairness Assessment Template - Example

## About
This example is intended as a simple illustration of the [Binary Classification Fairness Assessment Template](../templates/Template-BinaryClassificationAssessment.ipynb). It compares a Random Forest Classifier against fairness-aware alternative versions of that same classifier. For more information about the specific measures used, please see the [Measuring Fairness in Binary Classification Tutorial](../tutorials_and_examples/Tutorial-MeasuringFairnessInBinaryClassification.ipynb).

For simplicity, only two fairness-aware algorithms are compared in this notebook. However, several other fairness-aware models were tested during development. For a peek at that process, see [Supplemental - Models for Binary Classification Example](../tutorials_and_examples/Supplemental-ModelsForBinaryClassificationExample.ipynb).

## Example Contents

[Part 1](#part1) - Data Loading and Model Setup

[Part 2](#part2) - Fairness-Aware Models

[Part 3](#part3) - Model Comparison


In [1]:
from IPython.display import Markdown, HTML
from fairmlhealth import model_comparison as fhmc, reports, stratified_reports, tutorial_helpers as helpers
import numpy as np
import pandas as pd


# Functions and pointers to make this tutorial more colorful
ks_magenta = '#d00095'
ks_magenta_lt = '#ff05b8'
ks_purple = '#947fed'

import warnings
#warnings.filterwarnings('ignore', module='numpy')
warnings.filterwarnings('ignore', module='tensorflow')

----
# Load Data and Generate Baseline Model <a name="part1"></a>

## MIMIC-III

This example uses a simple data subset from the [MIMIC-III clinical database](https://mimic.physionet.org/gettingstarted/access/) to predict the length of ICU stay (LOS) for a set of encounters. MIMIC-III is a freely available database, however all users must pass a quick human subjects certification course. For the example, LOS is the total intensive care unit (ICU) time for a given hospital admission in patients 65 and above. The raw LOS value is then converted to a binary value specifying whether an admission's length of stay is greater than the sample mean. 

Note that the code below will automatically unzip and format all necessary data for these experiments from a raw download of MIMIC-III data (saving the formatted data in the same MIMIC folder). If you would like to run this example on your own, [follow these steps to be granted access to MIMIC III](https://mimic.physionet.org/gettingstarted/access/) and download the data.


## Data Subset

Data are imported at the encounter level with all additional patient identification dropped. Boolean diagnosis and procedure features are categorized through the Clinical Classifications Software system ([HCUP](https://www.hcup-us.ahrq.gov/toolssoftware/ccs/ccs.jsp)). All features other than age are one-hot encoded and prefixed with their variable type (e.g. "GENDER_", "ETHNICITY_").  


In [2]:
# path_to_mimic_data_folder = "[path to folder containing your MIMIC-III zip files]"
path_to_mimic_data_folder = "~/data/MIMIC"

In [3]:
# Load data and subset to ages 65+
df = helpers.load_mimic3_example(path_to_mimic_data_folder) 
df = df.loc[df['AGE'].ge(65), :]
df.drop('GENDER_F', axis=1, inplace=True) # Redundant with GENDER_M

# Generate a binary target flagging whether an observation's length_of_stay value is above or below the mean. 
mean_val = df['length_of_stay'].mean()
df['long_los'] = df['length_of_stay'].apply(lambda x: 1 if x > mean_val else 0)

display(df.head(2))

,ADMIT_ID,AGE,GENDER_M,ETHNICITY_AMERICAN INDIAN/ALASKA NATIVE,ETHNICITY_AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGNIZED TRIBE,ETHNICITY_ASIAN,ETHNICITY_ASIAN - ASIAN INDIAN,ETHNICITY_ASIAN - CAMBODIAN,ETHNICITY_ASIAN - CHINESE,ETHNICITY_ASIAN - FILIPINO,...,PROCEDURE_CCS_223,PROCEDURE_CCS_224,PROCEDURE_CCS_225,PROCEDURE_CCS_226,PROCEDURE_CCS_227,PROCEDURE_CCS_228,PROCEDURE_CCS_229,PROCEDURE_CCS_231,length_of_stay,long_los
0,1039187,65.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1.144444,0
1,1026095,70.0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,5.496528,0


## Split Data

In [4]:
from sklearn.model_selection import train_test_split

# Subset and Split Data
X = df.loc[:, [c for c in df.columns 
                if c not in ['ADMIT_ID', 'length_of_stay', 'long_los']]]
y = df.loc[:, ['long_los']]
splits = train_test_split(X, y, stratify=y, test_size=0.33, random_state=42)
X_train, X_test, y_train, y_test=splits

## Test Baseline

A Scikit-Learn Random Forest Classifier serves as our basis for comparison. Parameters were tuned using Scikit-Learn's GridSearch in the [Supplemental - Models for Binary Classification Example](../tutorials_and_examples/Supplemental-ModelsForBinaryClassificationExample.ipynb).

In [5]:
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier

In [6]:
# Set model parameters (currently set as default values, but defined here to be explicit)
rf_params = {'n_estimators': 1800, 'min_samples_split': 5, 'bootstrap': False}

# Train Model
rf_model = RandomForestClassifier(**rf_params)
rf_model.fit(X_train, y_train.iloc[:, 0])
y_pred_rf = rf_model.predict(X_test)

# display performance 
print("\n", "Random Forest Prediction Scores:", "\n", 
      classification_report(y_test, y_pred_rf, target_names=['LOS <= mean', 'LOS > mean']))


 Random Forest Prediction Scores: 
               precision    recall  f1-score   support

 LOS <= mean       0.77      0.89      0.83      4531
  LOS > mean       0.77      0.58      0.66      2873

    accuracy                           0.77      7404
   macro avg       0.77      0.74      0.74      7404
weighted avg       0.77      0.77      0.76      7404



In [7]:
# Generate comparison table (returned as a pandas dataframe)
meas = fhmc.measure_model(X_test, y_test, X_test['LANGUAGE_ENGL'], rf_model)
reports.flag(meas)

In [8]:
stratified_reports.classification_performance(X_test, y_test, y_pred_rf, features=['LANGUAGE_ENGL'])

,FEATURE,FEATURE VALUE,N OBS,POSITIVE CLASS RATE,POSITIVE PREDICTION RATE,ACCURACY,PRECISION,TPR,TNR,FPR,FNR,ROC_AUC,PR_AUC
0,ALL_FEATURES,ALL_VALUES,7404.0,0.3880,0.2897,0.7712,0.7748,0.5785,0.8934,0.1066,0.4215,0.5,0.3060
1,LANGUAGE_ENGL,0,3356.0,0.4195,0.3144,0.7589,0.7839,0.5874,0.8830,0.1170,0.4126,0.5,0.2902
2,LANGUAGE_ENGL,1,4048.0,0.3619,0.2693,0.7814,0.7661,0.5700,0.9013,0.0987,0.4300,0.5,0.3190


In [9]:
stratified_reports.classification_fairness(X_test, y_test, y_pred_rf, features=['LANGUAGE_ENGL'])

,FEATURE,FEATURE VALUE,PPV Ratio,TPR Ratio,FPR Ratio,TNR Ratio,FNR Ratio,PPV Difference,TPR Difference,FPR Difference,TNR Difference,FNR Difference,N OBS
0,LANGUAGE_ENGL,0,-0.0178,0.9704,0.8435,1.0207,1.0421,-0.0178,-0.0174,-0.0183,0.0183,0.0174,3356.0
1,LANGUAGE_ENGL,1,0.0178,1.0305,1.1856,0.9797,0.9596,0.0178,0.0174,0.0183,-0.0183,-0.0174,4048.0


----
# Fairness-Aware Models <a name="part2"></a>


## Fairlearn Models

The [Fairlearn](https://fairlearn.github.io/) package includes three [mitigation algorithms](https://fairlearn.github.io/user_guide/mitigation.html) designed to increase the fairness of an existing model relative to one of two user-specified fairness metrics. Both algorithms and metrics are listed in the cell below.

For more information about the specifics of these fairness metrics, see [Part 5 of the Measuring Fairness in Binary Classification Tutorial](../tutorials_and_examples/Tutorial-MeasuringFairnessInBinaryClassification.ipynb#part5).

In [ ]:
# Mitigation Algorithms
from fairlearn.reductions import GridSearch, ExponentiatedGradient

# Fairness Measures
from fairlearn.reductions import EqualizedOdds, DemographicParity 

### Fair ExponentiatedGradient

Fairlearn's ExponentiatedGradient is a wrapper that runs a constrained optimization using the Exponentiated Gradient approach on a binary classification model. It treats the prediction as a sequence of cost-sensitive classification problems, returning the solution with the smallest error (constrained by the metric of choice). This approach has been demonstrated to have minimal effect on model performance by some measures. [[Agarwal2018]](#Agarwal2018)

This approach is applicable to sensitive attributes that are either categorical or binary/Boolean. It can be used for classification problems only.

Note: solutions are not guaranteed for this approach.


In [ ]:
# Set seed for consistent results with Fairlearn's ExponentiatedGradient
np.random.seed(36)  

#### Fair ExponentiatedGradient Using Demographic Parity as Constraint

In [ ]:
eg_rfDP_model = ExponentiatedGradient(RandomForestClassifier(**rf_params), 
                                      constraints=DemographicParity()) 
eg_rfDP_model.fit(X_train, y_train,
                  sensitive_features=X_train['LANGUAGE_ENGL'])
y_pred_eg_rfDP = eg_rfDP_model.predict(X_test)

# display performance 
print("\n", "Prediction Scores:", "\n", 
      classification_report(y_test, y_pred_eg_rfDP, 
       target_names=['LOS <= mean', 'LOS > mean']))


#### Fair ExponentiatedGradient Using Equalized Odds as Constraint

In [ ]:
eg_rfEO_model = ExponentiatedGradient(RandomForestClassifier(**rf_params), 
                                      constraints=EqualizedOdds())  
eg_rfEO_model.fit(X_train, y_train, 
                  sensitive_features=X_train['LANGUAGE_ENGL'])
y_pred_eg_rfEO = eg_rfEO_model.predict(X_test)

# display performance 
print("\n", "Prediction Scores:", "\n", 
      classification_report(y_test, y_pred_eg_rfEO, 
       target_names=['LOS <= mean', 'LOS > mean']))


### Fair GridSearch

Fairlearn's GridSearch is a wrapper that runs a constrained optimization using the Grid Search approach  on a binary classification or a regression model. It treats the prediction as a sequence of cost-sensitive classification problems, returning the solution with the smallest error (constrained by the metric of choice). This approach has been demonstrated to have minimal effect on model performance by some measures [[Agarwal2018]](#Agarwal2018).

This approach is applicable to sensitive attributes that are binary/Boolean only. It can be used for either binary classification or regression problems.


#### Fair GridSearch Using Equalized Odds as Constraint

In [ ]:
# Train GridSearch
gs_rfEO_model = GridSearch(RandomForestClassifier(**rf_params),
                           constraints=EqualizedOdds(),
                           grid_size=45)

gs_rfEO_model.fit(X_train, y_train, 
                  sensitive_features = X_train['LANGUAGE_ENGL'])
y_pred_gs_rfEO = gs_rfEO_model.predict(X_test)

# display performance 
print("\n", "Prediction Scores:", "\n", 
      classification_report(y_test, y_pred_gs_rfEO, 
       target_names=['LOS <= mean', 'LOS > mean']))


#### Fair GridSearch Using Demographic Parity as Constraint

In [ ]:
# Train GridSearch
gs_rfDP_model = GridSearch(RandomForestClassifier(**rf_params),
                           constraints=DemographicParity(),
                           grid_size=45)

gs_rfDP_model.fit(X_train, y_train, 
                  sensitive_features=X_train['LANGUAGE_ENGL'])
y_pred_gs_rfDP = gs_rfDP_model.predict(X_test)

# display performance 
print("\n", "Prediction Scores:", "\n", 
      classification_report(y_test, y_pred_gs_rfDP, 
       target_names=['LOS <= mean', 'LOS > mean']))


----
# Model Comparison <a name="part3"></a>


## Set the Required Variables  

* X (NumPy array or similar pandas object): test data to be passed to the models to generate predictions. It's recommended that these be separate data from those used to train the model.

* y (NumPy array or similar pandas object): target data array corresponding to X. It is recommended that the target is not present in the test_data.

* models (list or dict-like): the set of trained models to be evaluated. Note that the dictionary keys are assumed as model names. If a list-like object is passed, the function will set model names relative to their index (i.e. "model_0", "model_1", etc.)

* protected_attr (NumPy array or similar pandas object): protected attributes corresponding to X, optionally also included in X. Note that values must currently be binary- or Boolean-type.


In [ ]:
X = X_test
y = y_test
protected_attr = X_test['LANGUAGE_ENGL']
models = {'rf_model': rf_model,
         'gs_rfEO_model': gs_rfEO_model, 'gs_rfDP_model': gs_rfDP_model,
         'eg_rfEO_model': eg_rfEO_model, 'eg_rfDP_model': eg_rfDP_model}
display("Models being compared in this example:", list(models.keys()))


## Comparison with the FairMLHealth Tool

The FairMLHealth model comparison tool generates a table of fairness measures that can be used to quickly compare the fairness-performance tradeoff for a set of fairness-aware models. 

Note that there is some additional formatting added to the cell below simply to add highlighting for this example.

In [ ]:
# Generate comparison table (returned as a pandas dataframe)
comparison = fhmc.compare_models(X, y, protected_attr, models)

# Here we determine the indices for equal odds measures so that we can highlight according
#    to those indices later
idx = pd.IndexSlice
eotag = idx[:, ['Equal Opportunity Difference', 'Equalized Odds Difference',
                 'Equalized Odds Ratio']
            ]
equal_odds = comparison.loc[eotag, :].index

# Here we return the flagged table as a pandas styler so we can also highlight 
#       measures of Equal Odds
reports.flag_suspicious(comparison, as_styler=True
       ).apply(lambda x: ['background-color:' + ks_magenta_lt 
                          if x.name in equal_odds else '' for i in x]
                , axis=1)



# References

<a name="Agarwal2018"></a>
Agarwal, A., Beygelzimer, A., Dudík, M., Langford, J., & Wallach, H. (2018). A reductions approach to fair classification. [rXiv preprint arXiv:1803.02453](https://arxiv.org/pdf/1803.02453.pdf).